### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasetsbb
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [13]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = n_users * n_movies # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    error = diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += diff**2
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [7]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)
# use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimization Statistics
Iterations | Mean Squared Error 


In [8]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.08153561   8.86691141  10.64724644  10.13103845]
 [  8.86201983   7.63556806   8.91569462   8.70450934]
 [  7.78146493   6.84510814   8.24128781   7.83898118]
 [  9.89441292   8.42307437   9.71280255   9.59537693]
 [  8.42930538   7.43860805   9.02986979   8.5359284 ]
 [  6.87095595   6.01916322   7.19125004   6.86824571]
 [  8.89896192   8.01390654   9.89037892   9.20403649]
 [  8.38735409   7.21862771   8.43803504   8.2184167 ]
 [  8.3262374    7.38158363   9.02424595   8.50481787]
 [  7.85700048   6.73075872   7.90715755   7.69961664]
 [  8.7789691    7.71642037   9.38629371   8.88451493]
 [  9.77521578   8.42646137   9.8899518    9.59744557]
 [  9.49616835   8.1188761    9.5136183    9.29668656]
 [  6.80325653   5.90252967   6.74068038   6.65320472]
 [  9.93896928   8.40100782  10.13607041   9.77205217]
 [  9.52929672   8.55798069  10.35017419   9.73966526]
 [  8.43606894   7.29706569   8.53904484   8.30594759]
 [  7.30173625   6.16381822   7.27105493   7.09570162]
 [  7.9168

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [9]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)
#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 


In [10]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[  9.99999678   9.99999857   9.99999939  10.000003  ]
 [  9.99998799   3.9999948    8.99999782  10.00001114]
 [  7.99999556   8.99999824   9.99999934   5.00000411]
 [  8.99999668   7.99999877   9.99999965  10.00000318]
 [ 10.0000008    5.0000006    9.00000043   8.99999939]
 [  6.00000025   4.00000027  10.00000024   5.99999987]
 [  9.00000217   8.00000112  10.00000063   8.99999812]
 [ 10.0000073    5.00000329   9.0000015    7.99999339]
 [  7.00000801   8.0000035   10.00000152   7.99999268]
 [  9.00000525   5.00000217   9.0000009    6.99999524]
 [  9.00000018   7.99999997   9.99999993   7.99999986]
 [  8.99999688   9.99999853   9.99999934   9.0000029 ]
 [  9.99999484   8.99999774   9.99999903   8.00000478]
 [  5.00000148   8.0000007    5.00000033   7.99999866]
 [  9.99999661   7.99999857   9.99999941  10.00000317]
 [  8.99999484   8.99999784   9.99999915  10.00000482]
 [  9.00000489   8.00000222   8.00000102   7.99999557]
 [ 10.00000241   8.00000109   1.00000052   9.99999785]
 [  5.0000

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [11]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [14]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 43.818216
2 		 15.746713
3 		 4.332458
4 		 2.826670
5 		 2.680351
6 		 2.633475
7 		 2.596485
8 		 2.558773
9 		 2.517589
10 		 2.471679
11 		 2.420176
12 		 2.362352
13 		 2.297597
14 		 2.225444
15 		 2.145629
16 		 2.058153
17 		 1.963346
18 		 1.861917
19 		 1.754978
20 		 1.644024
21 		 1.530865
22 		 1.417506
23 		 1.305987
24 		 1.198211
25 		 1.095781
26 		 0.999885
27 		 0.911244
28 		 0.830119
29 		 0.756378
30 		 0.689595
31 		 0.629152
32 		 0.574346
33 		 0.524462
34 		 0.478834
35 		 0.436874
36 		 0.398094
37 		 0.362104
38 		 0.328605
39 		 0.297379
40 		 0.268272
41 		 0.241179
42 		 0.216031
43 		 0.192774
44 		 0.171368
45 		 0.151767
46 		 0.133922
47 		 0.117771
48 		 0.103241
49 		 0.090245
50 		 0.078686
51 		 0.068461
52 		 0.059459
53 		 0.051569
54 		 0.044682
55 		 0.038689
56 		 0.033490
57 		 0.028989
58 		 0.025101
59 		 0.021745
60 		 0.018852
61 		 0.016359
62 		 0.014210
63 		 0.012357
64 		

In [15]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.43552131214534:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [16]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users

user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=20)
#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 37.332000
2 		 11.712541
3 		 3.757998
4 		 2.806035
5 		 2.673261
6 		 2.624015
7 		 2.588481
8 		 2.553944
9 		 2.516863
10 		 2.475786
11 		 2.429796
12 		 2.378141
13 		 2.320165
14 		 2.255321
15 		 2.183220
16 		 2.103700
17 		 2.016899
18 		 1.923335
19 		 1.823963
20 		 1.720190


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [20]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
# How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
# How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [28]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
